In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
https = 'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/'
url = requests.get(https).text

soup = BeautifulSoup(url, 'lxml')
#print(soup.prettify())



### Observations From Original Version

We observed that more detailed data was conducted on data from 1996 and onwards. We will separate those dataframes to more easily analyze.

In [4]:
#we create a list of all useable links, all_links
raw_links = soup.find_all('a')
all_links = []
for link in raw_links:
    if (link.get('href') not in raw_links):
        all_links.append(link.get('href'))

#we create a list of all links to detailed data, csv_links
csv_detailed_links = []        
for link in all_links:
    if((link[-6:-1] == 'csv.g') and link.startswith('StormEvents_details')):
        csv_detailed_links.append(link)

#we create a list of all links to fatalaty data, csv_links
csv_fatalities_links = []        
for link in all_links:
    if((link[-6:-1] == 'csv.g') and link.startswith('StormEvents_fatalities')):
        csv_fatalities_links.append(link)
        
#we create a list of all links to location data, csv_links
csv_location_links = []        
for link in all_links:
    if((link[-6:-1] == 'csv.g') and link.startswith('StormEvents_locations')):
        csv_location_links.append(link)        

In [5]:
any(ext in "StormEvents_details-ftp_v1.0_d1950_c20170120.csv.gz" for ext in [str(x) for x in range(1996,2019)])
# [x for x in range(10).astype(str)]

True

In [6]:
det_df_after95 = pd.DataFrame()
for i in [link for link in csv_detailed_links if (link[30:34] in [str(x) for x in range(1996,2019)])]:
    det_df_after95 = pd.concat([det_df_after95,pd.read_csv(https + i, index_col = 'EVENT_ID')])

#we'll run these later    
# fatalities_df = pd.DataFrame()
# for i in csv_fatalities_links:
#     fatalities_df = pd.concat([fatalities_df,pd.read_csv(https + i)])
    
# location_df = pd.DataFrame()
# for i in csv_location_links:
#     location_df = pd.concat([location_df,pd.read_csv(https + i)])

/Users/MacBookPro/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/MacBookPro/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/MacBookPro/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (29,34,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
det_df_after95.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,STATE,STATE_FIPS,YEAR,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
EVENT_ID,,,,,,,,,,,,,,,,,,,,,
5570068,199610,29,1710,199610,30,600,2052149,ILLINOIS,17.0,1996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Strong gradient winds followed behind a line o...,NaN,PDC
5570362,199610,21,900,199610,21,900,2052748,OKLAHOMA,40.0,1996,...,NaN,NaN,DURANT,NaN,NaN,NaN,NaN,Flash flood waters washed out a small bridge a...,NaN,PDC
5570363,199610,20,1915,199610,20,1915,2052749,OKLAHOMA,40.0,1996,...,NaN,NaN,EMPIRE CITY,34.42,-98.03,34.42,-98.03,Hail as large as softballs broke out car and t...,NaN,PDC
5537534,199601,23,400,199601,24,2200,2404440,CALIFORNIA,6.0,1996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A storm that originated in the Gulf of Alaska ...,NaN,PDC
5538413,199601,17,900,199601,18,1800,2404601,MINNESOTA,27.0,1996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A major winter storm brought blizzard conditio...,NaN,PDC


In [8]:
det_df_after95.shape

(1288174, 50)

In [9]:
#CHECKS TO MAKE SURE THERE ARE NO DUPLICATES. IT'S ALL GOOD!!!!

# det_df_after95.sort_index()

# idx = pd.Index(det_df_after95)
# idx.duplicated()

# from collections import Counter

# indexes = idx.duplicated()

# print(Counter(indexes).keys()) # equals to list(set(words))
# print(Counter(indexes).values()) # counts the elements' frequency

In [10]:
#removing any column that has more than 60% of data as NA
#first get a list of boolean columns that have more than 60% of data still intact
test = det_df_after95.isnull().sum() < .6*det_df_after95.shape[0]

In [11]:
# returns list of column names that have 60%
columns = [test.index[i] for i in range(len(test)) if test[i]]
columns

['BEGIN_YEARMONTH',
 'BEGIN_DAY',
 'BEGIN_TIME',
 'END_YEARMONTH',
 'END_DAY',
 'END_TIME',
 'EPISODE_ID',
 'STATE',
 'STATE_FIPS',
 'YEAR',
 'MONTH_NAME',
 'EVENT_TYPE',
 'CZ_TYPE',
 'CZ_FIPS',
 'CZ_NAME',
 'WFO',
 'BEGIN_DATE_TIME',
 'CZ_TIMEZONE',
 'END_DATE_TIME',
 'INJURIES_DIRECT',
 'INJURIES_INDIRECT',
 'DEATHS_DIRECT',
 'DEATHS_INDIRECT',
 'DAMAGE_PROPERTY',
 'DAMAGE_CROPS',
 'SOURCE',
 'MAGNITUDE',
 'BEGIN_RANGE',
 'BEGIN_AZIMUTH',
 'BEGIN_LOCATION',
 'END_RANGE',
 'END_AZIMUTH',
 'END_LOCATION',
 'BEGIN_LAT',
 'BEGIN_LON',
 'END_LAT',
 'END_LON',
 'EPISODE_NARRATIVE',
 'EVENT_NARRATIVE',
 'DATA_SOURCE']

In [12]:
detailed_df_remove_na = det_df_after95[columns]

In [13]:
detailed_df_remove_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1288174 entries, 5570068 to 743878
Data columns (total 40 columns):
BEGIN_YEARMONTH      1288174 non-null int64
BEGIN_DAY            1288174 non-null int64
BEGIN_TIME           1288174 non-null int64
END_YEARMONTH        1288174 non-null int64
END_DAY              1288174 non-null int64
END_TIME             1288174 non-null int64
EPISODE_ID           1288174 non-null int64
STATE                1288173 non-null object
STATE_FIPS           1288173 non-null float64
YEAR                 1288174 non-null int64
MONTH_NAME           1288174 non-null object
EVENT_TYPE           1288174 non-null object
CZ_TYPE              1288174 non-null object
CZ_FIPS              1288174 non-null int64
CZ_NAME              1288174 non-null object
WFO                  1288174 non-null object
BEGIN_DATE_TIME      1288174 non-null object
CZ_TIMEZONE          1288174 non-null object
END_DATE_TIME        1288174 non-null object
INJURIES_DIRECT      1288174 non-nu

In [14]:
detailed_df_remove_na.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,STATE,STATE_FIPS,YEAR,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
EVENT_ID,,,,,,,,,,,,,,,,,,,,,
5570068,199610,29,1710,199610,30,600,2052149,ILLINOIS,17.0,1996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Strong gradient winds followed behind a line o...,NaN,PDC
5570362,199610,21,900,199610,21,900,2052748,OKLAHOMA,40.0,1996,...,NaN,NaN,DURANT,NaN,NaN,NaN,NaN,Flash flood waters washed out a small bridge a...,NaN,PDC
5570363,199610,20,1915,199610,20,1915,2052749,OKLAHOMA,40.0,1996,...,NaN,NaN,EMPIRE CITY,34.42,-98.03,34.42,-98.03,Hail as large as softballs broke out car and t...,NaN,PDC
5537534,199601,23,400,199601,24,2200,2404440,CALIFORNIA,6.0,1996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A storm that originated in the Gulf of Alaska ...,NaN,PDC
5538413,199601,17,900,199601,18,1800,2404601,MINNESOTA,27.0,1996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A major winter storm brought blizzard conditio...,NaN,PDC


In [15]:
#prints the number of unique values in each column
[print(column, len(detailed_df_remove_na[column].unique())) for column in detailed_df_remove_na.columns]

BEGIN_YEARMONTH 272
BEGIN_DAY 31
BEGIN_TIME 1440
END_YEARMONTH 272
END_DAY 31
END_TIME 1440
EPISODE_ID 321700
STATE 69
STATE_FIPS 69
YEAR 23
MONTH_NAME 12
EVENT_TYPE 61
CZ_TYPE 4
CZ_FIPS 587
CZ_NAME 4980
WFO 125
BEGIN_DATE_TIME 641511
CZ_TIMEZONE 16
END_DATE_TIME 631514
INJURIES_DIRECT 146
INJURIES_INDIRECT 58
DEATHS_DIRECT 41
DEATHS_INDIRECT 14
DAMAGE_PROPERTY 2671
DAMAGE_CROPS 1131
SOURCE 73
MAGNITUDE 410
BEGIN_RANGE 154
BEGIN_AZIMUTH 18
BEGIN_LOCATION 46962
END_RANGE 148
END_AZIMUTH 21
END_LOCATION 47342
BEGIN_LAT 102309
BEGIN_LON 128255
END_LAT 109350
END_LON 138791
EPISODE_NARRATIVE 188565
EVENT_NARRATIVE 584010
DATA_SOURCE 3


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [16]:
#what I'm thinking now is that there is so much data already... 1.5million+ rows, it might be okay to just remove rows with na
detailed_df_remove_na.shape

(1288174, 40)

In [17]:
detailed_df_remove_na.isna().sum()

BEGIN_YEARMONTH           0
BEGIN_DAY                 0
BEGIN_TIME                0
END_YEARMONTH             0
END_DAY                   0
END_TIME                  0
EPISODE_ID                0
STATE                     1
STATE_FIPS                1
YEAR                      0
MONTH_NAME                0
EVENT_TYPE                0
CZ_TYPE                   0
CZ_FIPS                   0
CZ_NAME                   0
WFO                       0
BEGIN_DATE_TIME           0
CZ_TIMEZONE               0
END_DATE_TIME             0
INJURIES_DIRECT           0
INJURIES_INDIRECT         0
DEATHS_DIRECT             0
DEATHS_INDIRECT           0
DAMAGE_PROPERTY      506101
DAMAGE_CROPS         618333
SOURCE               113626
MAGNITUDE            624736
BEGIN_RANGE          695551
BEGIN_AZIMUTH        695551
BEGIN_LOCATION       472630
END_RANGE            695928
END_AZIMUTH          695928
END_LOCATION         472616
BEGIN_LAT            535987
BEGIN_LON            535986
END_LAT             

In [18]:
#removes rows that contain any na
# det_df = detailed_df_remove_na.dropna()

#testing data without dropping
det_df = detailed_df_remove_na.copy()

In [19]:
# Check how many rows are removed
# det_df.info()

Let's see which columns need cleaning...
Referencing: https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/Storm-Data-Export-Format.docx

EPISODE_ID: float --> object/int 
CZ_FIPS: float --> object

BEGIN_DATE_TIME: object --> datetime
CZ_TIMEZONE
END_DATE_TIME: object --> datetime

**DAMAGE_PROPERTY: object --> int/float
DAMAGE_CROPS: object --> int/float

## Cleaning Data
Converting Strings ending in 'K', 'M', 'B', etc to Floats

### Solution 1 - for loop

### Results
Super slow to compute with so many rows. Takes an hour or more per column...

In [20]:
#created function to clean strings ending in "K" and "M"
#input dataframe and column name
def clean_nums(df, column):
    for i in range(len(df[column])):
        if type(df.loc[i, column]) == float:
            pass
        elif df.loc[i, column] == 'K':
            df.set_value(i, column, 1000)
        elif df.loc[i,column].endswith('K'):
            df.loc[i, column] = df.loc[i, column].replace("K", "")
            df.set_value(i, column, (float(df.loc[i, column]) * 1000))
        elif df.loc[i,column].endswith('M'):
            df.loc[i, column] = df.loc[i, column].replace("M", "")
            df.set_value(i, column, (float(df.loc[i, column]) * 1000000))
        else:
            df.set_value(i, column, float(df.loc[i,column]))
    df[column] = df[column].astype(float)

# BELOW is the testcase
#--------------------------------------------
# test_case = det_df.head(2000)
# test_case = test_case.reset_index(drop=True)
# print(test_case['DAMAGE_CROPS'])
# clean_nums(test_case, 'DAMAGE_CROPS')
# print(test_case['DAMAGE_CROPS'])

### Solution 2 - map'ing
### Results
Much faster than the for loop, but still a bit cumbersome and repetitive. 

In [21]:
# df_new = det_df[['DAMAGE_PROPERTY']]
# part1 = df_new[df_new['DAMAGE_PROPERTY'].map(lambda x: x[-1] == 'K')]['DAMAGE_PROPERTY'].map(lambda x: float(x[:-1]) * 1000)
#part2 = df_new[df_new['DAMAGE_PROPERTY'].map(lambda x: x[-1] == 'M')]['DAMAGE_PROPERTY'].map(lambda x: float(x[:-1]) * 1000000)
#part3 = df_new[df_new['DAMAGE_PROPERTY'].map(lambda x: x[-1] == 'B')]['DAMAGE_PROPERTY'].map(lambda x: float(x[:-1]) * 1000000000)
#part4 = df_new[df_new['DAMAGE_PROPERTY'].map(lambda x: x[-1] != 'K' and x[-1] != 'M' and x[-1] != 'B')]['DAMAGE_PROPERTY']

### Solution 3 - function w/ map

In [22]:
def clean_me(df, column):
    ends = [x[-1] for x in df[column].unique() if type(x)== str]
    
    extensions = ['K','M', 'B', 'T']

    df_new = df.copy()

    df_na = df_new[df_new[column].isna()]
    df_na[column]=0
    df_na_col = df_na[column]
    df_nona = df_new[df_new[column].notna()]
    
    final_col = pd.DataFrame()
    
    errors = []
    try:
        for i in [ext for ext in set(ends)]:

            if i in extensions:
                multiplier = extensions.index(i)
                create_series = df_nona[df_nona[column].map(lambda x: x[-1] == i)][column].map(lambda x: float(x[:-1]) * 10** (3*(multiplier+1)))

            else:
                create_series = df_nona[df_nona[column].map(lambda x: x[-1] == i)][column].map(lambda x: float(x))


            final_col = pd.concat([final_col,create_series])
    except:
        print("error")
            
    final_col= pd.concat([final_col, df_na_col])
    final_col = final_col.astype(float)
    return final_col

In [23]:
# ends = [x[-1] for x in df_new['DAMAGE_CROPS'].unique() if type(x)== str]
# print(pd.Series(ends).value_counts())
# print(set(ends))



a = clean_me(det_df, 'DAMAGE_CROPS')
print(a.shape)
print(det_df['DAMAGE_CROPS'].shape)

/Users/MacBookPro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


error
(644969, 1)
(1288174,)


In [25]:
det_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1288174 entries, 5570068 to 743878
Data columns (total 40 columns):
BEGIN_YEARMONTH      1288174 non-null int64
BEGIN_DAY            1288174 non-null int64
BEGIN_TIME           1288174 non-null int64
END_YEARMONTH        1288174 non-null int64
END_DAY              1288174 non-null int64
END_TIME             1288174 non-null int64
EPISODE_ID           1288174 non-null int64
STATE                1288173 non-null object
STATE_FIPS           1288173 non-null float64
YEAR                 1288174 non-null int64
MONTH_NAME           1288174 non-null object
EVENT_TYPE           1288174 non-null object
CZ_TYPE              1288174 non-null object
CZ_FIPS              1288174 non-null int64
CZ_NAME              1288174 non-null object
WFO                  1288174 non-null object
BEGIN_DATE_TIME      1288174 non-null object
CZ_TIMEZONE          1288174 non-null object
END_DATE_TIME        1288174 non-null object
INJURIES_DIRECT      1288174 non-nu